In [ ]:
# Notebook from https://medium.com/@thakermadhav/build-your-own-rag-with-mistral-7b-and-langchain-97d0c92fa146
!pip install -q torch datasets flashrank
!pip install -q accelerate==0.21.0 \
                peft==0.4.0 \
                bitsandbytes==0.40.2 \
                transformers==4.31.0 \
                trl==0.4.7
!pip install -q scipy langchain transformers playwright html2text sentence_transformers faiss-gpu
!pip install -q --upgrade git+https://github.com/huggingface/transformers
!pip install -U tokenizers

In [ ]:
import os
import torch
from transformers import (
  AutoTokenizer, 
  AutoModelForCausalLM, 
  BitsAndBytesConfig,
  pipeline
)

from operator import itemgetter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from transformers import BitsAndBytesConfig
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
import arrow
import pandas as pd
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
import warnings
warnings.filterwarnings('ignore')
import re
import glob

VAR_TRANSLATION={'DISPOSPERQTHENCEINTE': "dispositif_special_rqth_maternite_senior",
 'MONTANT_CAVIARDE': "montant_caviarde",
 'TELETRAVAIL_FLEX_SANS_LIMITE': "teletravail_regulier_flexible_sans_limite",
 'TT_REG_NOMBRE_FORMULE': "nombre_formules_teletravail_regulier",
 'DUREE': "duree_application",
 'REVERS': "presence_clause_reversibilite",
 'ADAPT': "periode_adaptation",
 'TTREG': "teletravail_regulier",
 'TTOCA': "teletravail_occasionnel",
 'TTEXC': "teletravail_exceptionnel",
 'TTMOIS': "mention_teletravail_par_mois",
 'TTMOIS_NOMBRE': "nombre_jours_teletravail_mois",
 'TTTRIM': "mention_teletravail_par_trimestre",
 'TTTRIM_NOMBRE': "nombre_jours_teletravail_trimestre",
 'TTANNEE': "mention_teletravail_par_annuel",
 'TTANNEE_NOMBRE': "nombre_jours_teletravail_annuel",
 'TTSEM': "mention_teletravail_par_semaine",
 'TTSEM_NOMBRE': "nombre_jours_teletravail_semaine",
 'TTOCAMOIS': "occas_mention_teletravail_par_mois",
 'TTOCA_MOIS_NOMBRE': "occas_nombre_jours_teletravail_mois",
 'TTOCATRIM': "occas_mention_teletravail_par_trimestre",
 'TTOCA_TRIM_NOMBRE': "occas_nombre_jours_teletravail_trimestre",
 'TTOCAANNEE': "occas_mention_teletravail_par_annuel",
 'TTOCA_ANNEE_NOMBRE': "occas_nombre_jours_teletravail_annuel",
 'TTOCASEM': "occas_mention_teletravail_par_semaine",
 'TTOCA_SEM_NOMBRE': "occas_nombre_jours_teletravail_semaine",
 'TTPRESJOURMOIS': "mention_jour_presence_par_mois",
 'PRESJOURMOIS_NOMBRE': "nombre_jour_presence_par_mois",
 'TTPRESJOURTRIM': "mention_jour_presence_par_trimestre",
 'PRESJOURTRIM_NOMBRE': "nombre_jour_presence_par_trimestre",
 'TTPRESJOURANNEE': "mention_jour_presence_par_annuel",
 'PRESJOURANNEE_NOMBRE': "nombre_jour_presence_par_annuel",
 'TTPRESJOURSEM': "mention_jour_presence_par_semaine",
 'PRESJOURSEM_NOMBRE': "nombre_jour_presence_par_semaine",
 'COMPJOUR': "mention_indemnisation_journaliere",
 'COMPJOUR_NOMBRE': "indemnisation_journaliere",
 'COMPMOIS': "mention_indemnisation_mensuelle",
 'COMPMOIS_NOMBRE': "indemnisation_mensuelle",
 'COMPMOIS_BASE_NOMBRE': "indemnisation_base_mensuelle_pour_un_jour_par_semaine",
 'COMPTRIM': "mention_indemnisation_trimestrielle",
 'COMPTRIM_NOMBRE': "indemnisation_trimestrielle",
 'COMPANNEE': "mention_indemnisation_annuelle",
 'COMPANNEE_NOMBRE': "indemnisation_annuelle",
 'COMPSEM': "mention_indemnisation_semaine",
 'COMPSEM_NOMBRE': "indemnisation_semaine",
 'COMPE': "mention_indemnisation_equipement",
 'COMPE_NOMBRE': "indemnisation_equipement",
 'COMPS': "mention_indemnite_sujetion",
 'COMPS_NOMBRE': "montant_indemnite_sujetion",
 'EQUIPEMENT': "equipement_fourni"}

def guess_reponse_booleenne(reponse):
    return guess_reponse_booleenne_from_string(reponse["text"])

def guess_reponse_booleenne_from_string(str):
    reponse_booleenne=None
    if re.match("^(\n| )*(Oui|Yes)", str):
        reponse_booleenne=1
    elif re.match("^(\n| )*(Non|No)", str):
        reponse_booleenne=0
    return reponse_booleenne

def guess_reponse_durée(reponse):
    reponse_booleenne=None
    if "indéterminée" in reponse["text"].lower() :
        reponse_booleenne=0
    elif "déterminée" in reponse["text"].lower():
        reponse_booleenne=1
    return reponse_booleenne

def guess_reponse_nombre(reponse,pattern="nombre"):
    reponse_nombre=None
    REGEX=rf".*{pattern} ?= ?(\d+)"
    numbers=re.findall(REGEX,reponse["text"])
    if numbers:
        reponse_nombre=numbers[0]
    return reponse_nombre

model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    cache_dir=".",
)
model.config.pad_token_id = tokenizer.pad_token_id

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
prompt_template = """

### [ROLE] Role: Vous êtes un expert juridique des accords d'entreprise sans compétence de programmation informatique, seulement la lecture et la compréhension des textes. Tu ne connais aucun langage de programmation, ni les expressions régulières.


### [INST] Instructions: 

1. Votre travail consiste à peupler et mettre à jour une base des données nécessaires pour réaliser des études économétriques poussées sur le télétravail.
        Pour cela, vous devez prendre le texte d'un accord d'entreprise dans les documents suivants, dans lesquelles figurent les informations à jour que l'on vous fourni, et d'extraire précisément l'information spécifique dont vous avez besoin.
        Vous ne devez extraire que le nombre, et uniquement le nombre. Le nombre peut être écrit en lettres ou en chiffres. (exemple : deux ou 2 )
        Si vous ne trouvez pas la réponse dans le texte vous devez le dire et ne pas chercher à en fournir un autre nombre.
        Avant de répondre, vérifier que la réponse se trouve bien dans le texte indiqué.
        
2. Vous devez respecter les règles suivantes : 
        2.A - Déconnecter les liens entre les périodicités (semaine, mois, trimestre, année)! Faire comme si les semaines, les mois, les trimestres et les années sont des concepts indépendants : dans un mois, il y a donc 0 semaine ; dans un trimestre, 0 mois et 0 semaine; dans une année, 0 trimestre, 0 mois et 0 semaine !
        2.B - Ne faites aucune supposition ni d'hypothèses : déconnectez le module de raisonnement !
        2.C - Ne faites aucun calcul de proratisation : par exemple, s'il y a 10 jours de télétravail autorisés par an, alors on ne peut pas déduire le nombre de jour de télétravail autorisés par semaine.
        2.D - Il n'y a pas de lien entre l'effectif d'un service et le nombre de jour de télétravail, donc ne pas déduire un nombre de jour de télétravail !
        2.E - Si le document mentionne positivement le Télétravail à 100%, ou le full-remote, ET qu'il n'y a pas de jour de présence minimum sur site, alors 5 jours de télétravail est possible par semaine !
        2.F - Si le document mentionne par exemple trois jours de présence sur site et sans autre mention du nombre de jours de télétravail autorisé, alors tu ne peux pas donner de nombre de jour de télétravail par semaine a priori !
        2.G - Il y a cinq jours ouvrés dans une semaine, donc un nombre de jour de télétravail par semaine ne peut pas être strictement supérieur à cinq.
        2.H - Si tu donnes un nombre réel, ne donne pas la fraction mais un arrondi à l'entier inférieur !
        2.I - Si le nombre de jour de télétravail annuel est inférieur à 52, ce nombre de jour de télétravail concerne du télétravail occasionnel. S'il est supérieur ou égal à 52, ce nombre concerne du télétravail régulier.
        2.J - Si le nombre de jour de télétravail trimestriel est inférieur à 13, ce nombre de jour de télétravail concerne du télétravail occasionnel. S'il est supérieur ou égal à 13, ce nombre concerne du télétravail régulier.
        2.K - Si le nombre de jour de télétravail mensuelle est inférieur à 4, ce nombre de jour de télétravail concerne du télétravail occasionnel. S'il est supérieur ou égal à 4, ce nombre concerne du télétravail régulier.

3. Vous devez structurez votre réponse comme suit :
        * Mon extraction de deux paragraphes max pertinents provenant des documents à analyser qui répondent à la question (limité chacun à deux phrases max):
        * Ma réponse après raisonnement ci-dessous :
        * Mon raisonnenement :
        
### Documents à analyser: 
{context}

### QUESTION:
{question} [/INST]
"""

# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)


In [ ]:
prompt_template2 = """
[INST]

### Instruction: Vous devez répondre à la question suivante à partir des documents ci-dessous, en répondant par oui ou par non à la question posée.
{context}

### QUESTION:
{question} 

[/INST]
"""

# Create prompt from prompt template 
prompt2 = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template2,
)

# Create llm chain 
llm_chain2 = LLMChain(llm=mistral_llm, prompt=prompt2)


In [ ]:
PERIODICITE={ "SEM":"semaine"}

def process_model(rag_chain,rag_chain2,num_dossier):
    dict_reponse=dict()
    dict_reponse["num_dossier"]=num_dossier
    text=""
    for (k,v) in PERIODICITE.items():
        Q_TT=f"Est-ce qu'il y a au moins une mention à un nombre de jour de télétravail autorisé, recommandé, limité ou maximum, explicitement exprimé par {v} ?"
        reponse= rag_chain2.invoke(Q_TT)
        text+=f"Q_TT{k}:"+reponse["text"] + "\n"
        reponse_bool_int=guess_reponse_booleenne(reponse)
        dict_reponse[f"TT{k}"]=reponse_bool_int

        #if reponse_bool_int==1:
        Q_TT_NOMBRE=f"Quel est le nombre de jour maximum, recommandé ou limité, de télétravail par semaine ? En dehors des instructions et hors télétravail occasionnel ou ponctuel, est-ce qu'un paragraphe du document comporte le mot-clé {v} ou une notion de fréquence par {v} et concerne le nombre de jour de télétravail autorisé (ne compte pas les paragraphes qui relatent d'indemnisation par jour de télétravail, ni le télétravail ponctuel ou occasionnel)?"
        Q_TT_NOMBRE+=f" Si oui, depuis les informations du document, jusqu'à combien de jour possible (ou limite) de télétravail un employé/travailleur/salarié/collaborateur peut-il télétravailler régulièrement, jour exprimé seulement par {v} et explicitement dans un des paragraphes (attention, ne pas confondre avec les jours de présence obligatoire sur site)? "
        Q_TT_NOMBRE+=f" Si tu as plusieurs réponses parmi [ 0 , 0.5 , 1 , 1.5 , 2 , 2.5 , 3 , 3.5 , 4 , 4.5 , 5 ], affecte le maximum des jours de télétravail par {v} de ces éventuelles réponses dans une variable 'nombrejourdeteletravailmaxpar{v}'. Si tu ne trouves pas de valeur ou si les informations ne permettent pas de déterminer cette valeur, retourne nombrejourdeteletravailmaxpar{v} à 0"
        Q_TT_NOMBRE+=f" Si le télétravail à 100% ou le full-remote est autorisé ou fortement positivement évoqué ET qu'il n'y a pas de jour de présence minimum sur site, alors affecte à la variable nombrejourdeteletravailmaxpar{v} la valeur 5."
        Q_TT_NOMBRE+=f" Ta réponse devra impérativement comporter la chaîne de caractère 'nombrejourdeteletravailmaxpar{v}=' suivi du nombre demandé, en un seul exemplaire, dans la partie 'Ma réponse'."
        reponse= rag_chain.invoke({"question":Q_TT_NOMBRE,"previous_answer":reponse["text"]})
        text+=f"Q_TT{k}_NOMBRE:"+reponse["text"] + "\n"
        dict_reponse[f"TT{k}_NOMBRE"]=guess_reponse_nombre(reponse,pattern=f"nombrejourdeteletravailmaxpar{v}")

        Q_TTPRESJOUR=f"Est-ce qu'il y a au moins une mention à un nombre minimum de jour de présence sur site obligatoire, explicitement exprimé par {v} ?"
        reponse= rag_chain2.invoke(Q_TTPRESJOUR)
        text+=f"Q_TTPRESJOUR{k}:"+reponse["text"] + "\n"
        reponse_bool_int=guess_reponse_booleenne(reponse)
        dict_reponse[f"TTPRESJOUR{k}"]=reponse_bool_int

        #if reponse_bool_int==1:
        Q_TTPRESJOUR_NOMBRE=f"Quel est le nombre de jour minimum obligatoire sur site par semaine ? Affecte la valeur dans une variable nombrejourobligatoiresursite{v}."
        Q_TTPRESJOUR_NOMBRE+=f" Ta réponse devra impérativement comporter la chaîne de caractère 'nombrejourobligatoiresursite{v}=' suivi du nombre demandé, en un seul exemplaire, dans la partie 'Ma réponse'."
        reponse= rag_chain.invoke({"question":Q_TTPRESJOUR_NOMBRE,"previous_answer":reponse["text"]})
        text+=f"Q_TTPRESJOUR{k}_NOMBRE:"+reponse["text"] + "\n"
        dict_reponse[f"TTPRESJOUR{k}_NOMBRE"]=guess_reponse_nombre(reponse,pattern=f"nombrejourobligatoiresursite{v}")
    return dict_reponse,text

In [ ]:
PARQUET_FILE="Accords/teletravail_acemo_public.parquet"
df_init=pd.read_parquet(PARQUET_FILE)
PARQUET_FILE_COMPARE="Accords/Donnees_Mathilde_Pesenti_TT_Pour_DEE_v2_def.parquet"
already_done=glob.glob('Accords/Acemo/*_response.txt')

def get_numdossier(str):
    return str.split("/")[-1].split("_")[0]

for (index,(num_dossier,accord)) in df_init.iterrows():
    if num_dossier not in map(get_numdossier,already_done):
        print(index,num_dossier)
        with open(f"{num_dossier}.txt","w", encoding="utf-8") as file:
            file.write(accord)
    
        loader = TextLoader(f"{num_dossier}.txt", encoding='utf8')
        docs=loader.load()
        
        # Chunk text
        text_splitter = CharacterTextSplitter(chunk_size=400)
        chunked_documents = text_splitter.split_documents(docs)
        
        # Load chunked documents into the FAISS index
        db = FAISS.from_documents(chunked_documents, 
                                  HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"))
         
        retriever = db.as_retriever(search_kwargs={"k": 10})

        compressor = FlashrankRerank(model="ms-marco-MultiBERT-L-12",top_n=10)
        compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor, base_retriever=retriever
        )

        
        rag_chain = ( 
         {"context": itemgetter("previous_answer") | retriever,
            "question": itemgetter("question")}
            | llm_chain
        )
        rag_chain2 = ( 
         {"context": compression_retriever, "question": RunnablePassthrough()}
            | llm_chain2
        )
        dict_reponse,text=process_model(rag_chain,rag_chain2,num_dossier)
        with open(f"Accords/Acemo/{num_dossier}_response.txt","w", encoding="utf-8") as file:
            file.write(text)
            print(text)
        df2=pd.DataFrame.from_dict(dict_reponse,orient="index")
        df2=df2.transpose().rename(columns=VAR_TRANSLATION)
        df2.to_csv(f"Accords/Acemo/{num_dossier}.csv")
        df2.to_parquet(f"Accords/Acemo/{num_dossier}.parquet")

In [ ]:
data_files=glob.glob('Accords/Acemo/*.parquet')
df = pd.concat((pd.read_parquet(f, engine = 'pyarrow') for f in data_files))
df=df.set_index("num_dossier")
df=df[df.nombre_jours_teletravail_semaine.values!=[None]]
df2=pd.read_parquet(f"{PARQUET_FILE_COMPARE}")
df2=df2[["N..Dossier","nombre_jours_teletravail_semaine","mention_teletravail_par_semaine"]].set_index("N..Dossier")
df_merge=df2.merge(df,how="left",left_index=True, right_index=True)
df_merge.nombre_jours_teletravail_semaine_y=df_merge.nombre_jours_teletravail_semaine_y.astype("float64")
df_merge=df_merge.fillna(0.0)
df_merge["diff"]=(df_merge.nombre_jours_teletravail_semaine_x!=df_merge.nombre_jours_teletravail_semaine_y)
print("global accuracy",1-(df_merge["diff"].sum()/df_merge.shape[0]))

df_rempli=df_merge[df_merge.nombre_jours_teletravail_semaine_x!=0.0]
nb_correct,total =df_rempli.shape[0]-df_rempli["diff"].sum(),df_rempli.shape[0]
print("accuracy on non null value",1-(df_rempli["diff"].sum()/df_rempli.shape[0]),f"; nombre correct : {nb_correct} sur {total}")

df_rempli=df_merge[df_merge.nombre_jours_teletravail_semaine_x==0.0]
nb_correct,total =df_rempli.shape[0]-df_rempli["diff"].sum(),df_rempli.shape[0]
print("accuracy on null value",1-(df_rempli["diff"].sum()/df_rempli.shape[0]),f"; nombre correct : {nb_correct} sur {total}")

In [ ]:
data_files=glob.glob('*.parquet')
df = pd.concat((pd.read_parquet(f, engine = 'pyarrow') for f in data_files))
df=df.set_index("num_dossier")
df=df[df.nombre_jours_teletravail_semaine.values!=[None]]
df2=pd.read_parquet(f"{PARQUET_FILE_COMPARE}")
df2=df2[["N..Dossier","nombre_jours_teletravail_semaine","mention_teletravail_par_semaine"]].set_index("N..Dossier")
df_merge=df2.merge(df,how="left",left_index=True, right_index=True)
df_merge.nombre_jours_teletravail_semaine_y=df_merge.nombre_jours_teletravail_semaine_y.astype("float64")
df_merge=df_merge.fillna(0.0)
df_merge["diff"]=(df_merge.nombre_jours_teletravail_semaine_x!=df_merge.nombre_jours_teletravail_semaine_y)
print("global accuracy",1-(df_merge["diff"].sum()/df_merge.shape[0]))

df_rempli=df_merge[df_merge.nombre_jours_teletravail_semaine_x!=0.0]
nb_correct,total =df_rempli.shape[0]-df_rempli["diff"].sum(),df_rempli.shape[0]
print("accuracy on non null value",1-(df_rempli["diff"].sum()/df_rempli.shape[0]),f"; nombre correct : {nb_correct} sur {total}")

df_rempli=df_merge[df_merge.nombre_jours_teletravail_semaine_x==0.0]
nb_correct,total =df_rempli.shape[0]-df_rempli["diff"].sum(),df_rempli.shape[0]
print("accuracy on null value",1-(df_rempli["diff"].sum()/df_rempli.shape[0]),f"; nombre correct : {nb_correct} sur {total}")

In [ ]:
contingency_matrix = pd.crosstab(df_merge['mention_teletravail_par_semaine_y'], df_merge['mention_teletravail_par_semaine_x'])
contingency_matrix

In [ ]:
df_merge[df_merge.mention_teletravail_par_semaine_x!=df_merge.mention_teletravail_par_semaine_y][['mention_teletravail_par_semaine_x','mention_teletravail_par_semaine_y']]